# Funzioni d'utilità

In [1]:
import os
import shutil
import json
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
def remove_and_make_dir(path):
    if os.path.exists(path):
        shutil.rmtree(path)
        os.mkdir(path)
    else:
        os.mkdir(path)

In [3]:
def reformat_json(path):
    with open(path, "wr") as f:
        tmp = json.loads(f.read())
        json_obj = json.dumps(tmp, indent=4)
        f.write(json_obj)
        f.close()

In [4]:
def jaccard(set1, set2):
    intersection_size = len(set1 & set2)
    union_size = len(set1 | set2)
    if union_size > 0:
        return intersection_size / union_size
    else:
        return 0

In [38]:
from itertools import combinations
import pandas as pd
from Levenshtein import distance

In [49]:
def extract_schema_paolo(directory, name, final_schema_directory, t_jaccard, t_edit, format, head):

    result_jaccard = {}
    for filename in os.listdir(directory):

        print(filename)
        file = os.path.join(directory, filename)
        df = pd.read_csv(file, encoding="utf-8", low_memory=False)
        df = df.head(head)
        
        columns = list(df.columns)
        n_cols = len(columns)
        combs = combinations(range(n_cols), 2)
        print("Numbero di confronti: ", n_cols * (n_cols-1) // 2)
        k = 1
                
        for c1, c2 in combs:
            
            cc1 = columns[c1]
            cc2 = columns[c2]
            
            tmp1 = cc1.split("-")[-1]
            tmp2 = cc2.split("-")[-1]
            relative_edit_distance = distance(tmp1, tmp2) / max(len(tmp1), len(tmp2))
            print("EDIT", k, cc1, cc2, relative_edit_distance)
            if relative_edit_distance < t_edit:
                result_jaccard.setdefault(cc1, set()).add(cc2)
                result_jaccard.setdefault(cc2, set()).add(cc1)
                
            else:
                set1 = set(df.iloc[:, c1])
                set2 = set(df.iloc[:, c2])

                s = jaccard(set1, set2)
                print("JACCARD", k, cc1, cc2, s)
                if s > t_jaccard:
                    result_jaccard.setdefault(cc1, set()).add(cc2)
                    result_jaccard.setdefault(cc2, set()).add(cc1)
            
            k += 1
        
        del df

    s = set()
    for main_token, tokens in result_jaccard.items():
        tmp = set()
        tmp.add(main_token)
        tmp.update(tokens)
        s.add(frozenset(tmp))

    dict_synonyms = {}
    for tokens in s:
        tokens = set(tokens)
        tmp = list(tokens)[0].split("-")[-1]
        if tmp not in dict_synonyms.keys() and tokens != None:
            dict_synonyms[tmp] = tokens
        elif dict_synonyms[tmp] != None:
            dict_synonyms[tmp] = dict_synonyms[tmp].update(tokens)
            
    dict_synonyms = {k:list(v) for k,v in dict_synonyms.items() if v!=None}
            
    json_obj = json.dumps(dict_synonyms, indent=4)
    f = open(final_schema_directory + name + format, "w")
    f.write(json_obj)
    f.close()

    print("\n\n")

# Schemi mediati cluster

In [7]:
from task_1 import make_cluster_columns_cluster
#from task_2 import schema_cluster
from task_4 import make_schema_cluster

In [8]:
clusters_datasets_base_path = "..\\..\\Dataset\\ClusterParsed\\"
clusters_info_path = "..\\DatasetSchemaMatch\\"
clusters_filename_synonym = "dic_pre_val.txt"

clusters = ".\\clusters\\"
clusters_columns = ".\\clusters\\columns\\"
clusters_dictionary = ".\\clusters\\dictionary\\"
clusters_final_dictionary = ".\\clusters\\final_dictionary\\"
clusters_schema = ".\\clusters\\schema\\"

In [ ]:
#remove_and_make_dir(clusters)

### TASK 1

In [ ]:
print("\nMake cluster columns\n")
remove_and_make_dir(clusters_columns)
make_cluster_columns_cluster(clusters_datasets_base_path, clusters_info_path, clusters_filename_synonym, clusters_columns)

### TASK 2

In [51]:
def schema_cluster(directory, final_schema_directory, t_jaccard, t_edit, format, head):
    for dirname in os.listdir(directory):
        print(dirname)
        cluster = os.path.join(directory, dirname)
        extract_schema_paolo(cluster, dirname, final_schema_directory, t_jaccard, t_edit, format, head)

In [ ]:
print("\nExtract schema\n")
remove_and_make_dir(clusters_dictionary)
start = time.time()
schema_cluster(clusters_columns, clusters_dictionary, 0.1, 0.5, ".txt", 10000)
end = time.time()
print("\nTempo di esecuzione: "+str(end-start)+" (s)"+"\n")


Extract schema

ambitiobox
foundation_year.csv
Numbero di confronti:  0
industry.csv
Numbero di confronti:  3
EDIT 1 06-MarScoToc-industry 06-MarScoToc-headquarter 0.7272727272727273
JACCARD 1 06-MarScoToc-industry 06-MarScoToc-headquarter 0.0005173305742369374
EDIT 2 06-MarScoToc-industry 06-MarScoToc-name 0.875
JACCARD 2 06-MarScoToc-industry 06-MarScoToc-name 0.00021748586341887777
EDIT 3 06-MarScoToc-headquarter 06-MarScoToc-name 0.8181818181818182
JACCARD 3 06-MarScoToc-headquarter 06-MarScoToc-name 0.0
ownership.csv
Numbero di confronti:  0



ariregister
address.csv
Numbero di confronti:  0
capital.csv
Numbero di confronti:  0
id.csv
Numbero di confronti:  0
legal_form.csv
Numbero di confronti:  0
registration_date.csv
Numbero di confronti:  3
EDIT 1 09-wissel-registration_date 09-wissel-deletion_date 0.4117647058823529
EDIT 2 09-wissel-registration_date 09-wissel-code 0.8235294117647058
JACCARD 2 09-wissel-registration_date 09-wissel-code 0.0
EDIT 3 09-wissel-deletion_date 09-

JACCARD 18 03-gren-market_capitalization_usd 09-wissel-earnings 0.08837023738280471
EDIT 19 03-gren-market_capitalization_usd 02-GioPonSpiz-pricecap 0.84
JACCARD 19 03-gren-market_capitalization_usd 02-GioPonSpiz-pricecap 0.21956617972554227
EDIT 20 03-gren-market_capitalization_usd 09-wissel-revenue 0.84
JACCARD 20 03-gren-market_capitalization_usd 09-wissel-revenue 0.1797024952015355
EDIT 21 03-gren-market_capitalization_usd 01-DDD-marketcap 0.64
JACCARD 21 03-gren-market_capitalization_usd 01-DDD-marketcap 0.21814159292035398
EDIT 22 03-gren-market_capitalization_usd 04-iGMM-marketcap 0.64
JACCARD 22 03-gren-market_capitalization_usd 04-iGMM-marketcap 0.21168369613505109
EDIT 23 03-gren-market_capitalization_usd 05-MalPatSaj-marketcap 0.64
JACCARD 23 03-gren-market_capitalization_usd 05-MalPatSaj-marketcap 0.22192028985507245
EDIT 24 03-gren-market_capitalization_usd 09-wissel-marketcap 0.64
JACCARD 24 03-gren-market_capitalization_usd 09-wissel-marketcap 0.20797101449275363
EDIT 25

JACCARD 71 00-avengers-share_price 00-avengers-rank 0.0
EDIT 72 00-avengers-share_price 01-DDD-rank 0.9090909090909091
JACCARD 72 00-avengers-share_price 01-DDD-rank 0.0
EDIT 73 00-avengers-share_price 04-iGMM-rank 0.9090909090909091
JACCARD 73 00-avengers-share_price 04-iGMM-rank 0.0
EDIT 74 00-avengers-share_price 06-MarScoToc-rank 0.9090909090909091
JACCARD 74 00-avengers-share_price 06-MarScoToc-rank 0.0001015228426395939
EDIT 75 00-avengers-share_price 04-iGMM-change_1day 0.7272727272727273
JACCARD 75 00-avengers-share_price 04-iGMM-change_1day 0.00020470829068577277
EDIT 76 01-DDD-share_price 06-MarScoToc-share_price 0.0
EDIT 77 01-DDD-share_price 09-wissel-share_price 0.0
EDIT 78 01-DDD-share_price 01-DDD-change1y 0.8181818181818182
JACCARD 78 01-DDD-share_price 01-DDD-change1y 0.0
EDIT 79 01-DDD-share_price 02-GioPonSpiz-price 0.5454545454545454
JACCARD 79 01-DDD-share_price 02-GioPonSpiz-price 0.29858253315043437
EDIT 80 01-DDD-share_price 03-gren-price 0.5454545454545454
JACC

JACCARD 4 00-avengers-employees 00-avengers-founded 0.0
EDIT 5 00-avengers-employees 04-iGMM-founded 0.8888888888888888
JACCARD 5 00-avengers-employees 04-iGMM-founded 0.0
EDIT 6 00-avengers-employees 10-DeBiGa-founded 0.8888888888888888
JACCARD 6 00-avengers-employees 10-DeBiGa-founded 0.0008488964346349745
EDIT 7 04-iGMM-employees 08-slytherin-employees 0.0
EDIT 8 04-iGMM-employees 10-DeBiGa-employees 0.0
EDIT 9 04-iGMM-employees 00-avengers-founded 0.8888888888888888
JACCARD 9 04-iGMM-employees 00-avengers-founded 0.0
EDIT 10 04-iGMM-employees 04-iGMM-founded 0.8888888888888888
JACCARD 10 04-iGMM-employees 04-iGMM-founded 0.0
EDIT 11 04-iGMM-employees 10-DeBiGa-founded 0.8888888888888888
JACCARD 11 04-iGMM-employees 10-DeBiGa-founded 0.0007961783439490446
EDIT 12 08-slytherin-employees 10-DeBiGa-employees 0.0
EDIT 13 08-slytherin-employees 00-avengers-founded 0.8888888888888888
JACCARD 13 08-slytherin-employees 00-avengers-founded 0.0
EDIT 14 08-slytherin-employees 04-iGMM-founded 0

Numbero di confronti:  66
EDIT 1 00-avengers-name 02-GioPonSpiz-name 0.0
EDIT 2 00-avengers-name 03-gren-name 0.0
EDIT 3 00-avengers-name 04-iGMM-name 0.0
EDIT 4 00-avengers-name 05-MalPatSaj-name 0.0
EDIT 5 00-avengers-name 06-MarScoToc-name 0.0
EDIT 6 00-avengers-name 07-silvestri-name 0.0
EDIT 7 00-avengers-name 08-slytherin-name 0.0
EDIT 8 00-avengers-name 10-DeBiGa-name 0.0
EDIT 9 00-avengers-name 04-iGMM-headquarters 0.8333333333333334
JACCARD 9 00-avengers-name 04-iGMM-headquarters 0.0006983240223463687
EDIT 10 00-avengers-name 08-slytherin-headquarters 0.8333333333333334
JACCARD 10 00-avengers-name 08-slytherin-headquarters 0.0
EDIT 11 00-avengers-name 10-DeBiGa-headquarters 0.8333333333333334
JACCARD 11 00-avengers-name 10-DeBiGa-headquarters 0.0006983240223463687
EDIT 12 02-GioPonSpiz-name 03-gren-name 0.0
EDIT 13 02-GioPonSpiz-name 04-iGMM-name 0.0
EDIT 14 02-GioPonSpiz-name 05-MalPatSaj-name 0.0
EDIT 15 02-GioPonSpiz-name 06-MarScoToc-name 0.0
EDIT 16 02-GioPonSpiz-name 07-

EDIT 16 08-slytherin-revenue 07-silvestri-revenue_2022 0.4166666666666667
EDIT 17 08-slytherin-revenue 05-MalPatSaj-assets 0.8571428571428571
JACCARD 17 08-slytherin-revenue 05-MalPatSaj-assets 0.044054747647562016
EDIT 18 08-slytherin-revenue 05-MalPatSaj-sales 0.8571428571428571
JACCARD 18 08-slytherin-revenue 05-MalPatSaj-sales 0.11144130757800892
EDIT 19 07-silvestri-revenue_2022 05-MalPatSaj-assets 0.9166666666666666
JACCARD 19 07-silvestri-revenue_2022 05-MalPatSaj-assets 0.07813733228097869
EDIT 20 07-silvestri-revenue_2022 05-MalPatSaj-sales 0.9166666666666666
JACCARD 20 07-silvestri-revenue_2022 05-MalPatSaj-sales 0.584375
EDIT 21 05-MalPatSaj-assets 05-MalPatSaj-sales 0.6666666666666666
JACCARD 21 05-MalPatSaj-assets 05-MalPatSaj-sales 0.12342819324950365
name.csv
Numbero di confronti:  3
EDIT 1 05-MalPatSaj-name 07-silvestri-name 0.0
EDIT 2 05-MalPatSaj-name 08-slytherin-name 0.0
EDIT 3 07-silvestri-name 08-slytherin-name 0.0



ft
absolute_growth_rate_pct.csv
Numbero di con

### TASK 4

In [ ]:
print("\nMake schemas\n")
remove_and_make_dir(clusters_schema)
make_schema_cluster(clusters_datasets_base_path, clusters_final_dictionary, clusters_schema, ".txt")

# Schema mediato finale

In [ ]:
from task_1 import make_cluster_columns_final
#from task_2 import schema_final
from task_4 import make_schema_final

In [45]:
final_datasets_base_path = ".\\clusters\\schema\\"
final_info_path = ".\\files_matching\\files_vari"
final_filename_synonym = "dic_pre_val.txt"

final_schema = ".\\final_schema\\"
final_schema_columns = ".\\final_schema\\columns\\"
final_schema_dictionary = ".\\final_schema\\dictionary\\"
final_schema_final_dictionary = ".\\final_schema\\final_dictionary\\"
final_schema_schema = ".\\final_schema\\schema\\"

In [ ]:
#remove_and_make_dir(final_schema)

### TASK 1

In [ ]:
print("\nMake cluster columns\n")
remove_and_make_dir(final_schema_columns)
make_cluster_columns_final(final_datasets_base_path, final_info_path, final_filename_synonym, final_schema_columns)

### TASK 2

In [53]:
def schema_final(directory, final_schema_directory, t_jaccard, t_edit, format, head):
    extract_schema_paolo(directory, "finalschema", final_schema_directory, t_jaccard, t_edit, format, head)

In [55]:
print("\nExtract schema\n")
remove_and_make_dir(final_schema_dictionary)
start = time.time()
schema_final(final_schema_columns, final_schema_dictionary, 0.1, 0.5, ".txt", 15000)
end = time.time()
print("\nTempo di esecuzione: "+str(end-start)+" (s)"+"\n")


Extract schema

address.csv
Numbero di confronti:  6
EDIT 1 infoclipper-address_name globaldata-address 0.4166666666666667
EDIT 2 infoclipper-address_name hithorizons-address 0.4166666666666667
EDIT 3 infoclipper-address_name govuk-office_address 0.8571428571428571
JACCARD 3 infoclipper-address_name govuk-office_address 0.00024789291026276647
EDIT 4 globaldata-address hithorizons-address 0.0
EDIT 5 globaldata-address govuk-office_address 0.5
JACCARD 5 globaldata-address govuk-office_address 0.000152114390021296
EDIT 6 hithorizons-address govuk-office_address 0.5
JACCARD 6 hithorizons-address govuk-office_address 0.0005431830526887561
categories.csv
Numbero di confronti:  136
EDIT 1 wikipedia-notes disfold-sector 0.8333333333333334
JACCARD 1 wikipedia-notes disfold-sector 0.0036978341257263604
EDIT 2 wikipedia-notes ft-sector 0.8333333333333334
JACCARD 2 wikipedia-notes ft-sector 0.009404388714733543
EDIT 3 wikipedia-notes valuetoday-sector 0.8333333333333334
JACCARD 3 wikipedia-notes 

JACCARD 90 ariregister-industry companiesmarketcap-categories 0.0
EDIT 91 ariregister-industry disfold-categories 0.9
JACCARD 91 ariregister-industry disfold-categories 0.0
EDIT 92 cbinsights-industry disfold-industry 0.0
EDIT 93 cbinsights-industry forbes-industry 0.0
EDIT 94 cbinsights-industry globaldata-industry 0.0
EDIT 95 cbinsights-industry hithorizons-industry 0.0
EDIT 96 cbinsights-industry teamblind-industry 0.0
EDIT 97 cbinsights-industry valuetoday-industry 0.0
EDIT 98 cbinsights-industry wikipedia-industry 0.0
EDIT 99 cbinsights-industry companiesmarketcap-categories 0.9
JACCARD 99 cbinsights-industry companiesmarketcap-categories 0.000481000481000481
EDIT 100 cbinsights-industry disfold-categories 0.9
JACCARD 100 cbinsights-industry disfold-categories 0.002036659877800407
EDIT 101 disfold-industry forbes-industry 0.0
EDIT 102 disfold-industry globaldata-industry 0.0
EDIT 103 disfold-industry hithorizons-industry 0.0
EDIT 104 disfold-industry teamblind-industry 0.0
EDIT 10

JACCARD 30 globaldata-headquarters ariregister-ownership 0.0
EDIT 31 teamblind-headquarters wikipedia-headquarters 0.0
EDIT 32 teamblind-headquarters disfold-headquarters_continent 0.45454545454545453
EDIT 33 teamblind-headquarters valuetoday-headquarters_continent 0.45454545454545453
EDIT 34 teamblind-headquarters disfold-headquarters_country 0.4
EDIT 35 teamblind-headquarters cbinsights-city 0.9166666666666666
JACCARD 35 teamblind-headquarters cbinsights-city 0.002932551319648094
EDIT 36 teamblind-headquarters infoclipper-city 0.9166666666666666
JACCARD 36 teamblind-headquarters infoclipper-city 0.0006954102920723226
EDIT 37 teamblind-headquarters ambitiobox-ownership 0.9166666666666666
JACCARD 37 teamblind-headquarters ambitiobox-ownership 0.0023923444976076554
EDIT 38 teamblind-headquarters ariregister-ownership 0.9166666666666666
JACCARD 38 teamblind-headquarters ariregister-ownership 0.0
EDIT 39 wikipedia-headquarters disfold-headquarters_continent 0.45454545454545453
EDIT 40 wik

JACCARD 75 hithorizons-country wikipedia-headquarters 0.0007710100231303007
EDIT 76 infoclipper-country valuetoday-country 0.0
EDIT 77 infoclipper-country disfold-headquarters_continent 0.8636363636363636
JACCARD 77 infoclipper-country disfold-headquarters_continent 0.010752688172043012
EDIT 78 infoclipper-country valuetoday-headquarters_continent 0.8636363636363636
JACCARD 78 infoclipper-country valuetoday-headquarters_continent 0.1
EDIT 79 infoclipper-country disfold-headquarters_country 0.65
JACCARD 79 infoclipper-country disfold-headquarters_country 0.002531645569620253
EDIT 80 infoclipper-country infoclipper-state 0.8571428571428571
JACCARD 80 infoclipper-country infoclipper-state 0.08163265306122448
EDIT 81 infoclipper-country disfold-headquarters 0.75
JACCARD 81 infoclipper-country disfold-headquarters 0.0005230125523012552
EDIT 82 infoclipper-country globaldata-headquarters 0.75
JACCARD 82 infoclipper-country globaldata-headquarters 0.03529411764705882
EDIT 83 infoclipper-count

EDIT 47 ft-employees_number_2017 disfold-rank 0.9523809523809523
JACCARD 47 ft-employees_number_2017 disfold-rank 0.12151394422310757
EDIT 48 ft-employees_number_2017 valuetoday-rank 0.9523809523809523
JACCARD 48 ft-employees_number_2017 valuetoday-rank 0.008
EDIT 49 ft-employees_number_2017 ft-revenue_2017_eu 0.7142857142857143
JACCARD 49 ft-employees_number_2017 ft-revenue_2017_eu 0.0004710315591144607
EDIT 50 ft-employees_number_2017 valuetoday-employees_number 0.23809523809523808
EDIT 51 ft-employees_number_2017 globaldata-number_of_employees 0.9047619047619048
JACCARD 51 ft-employees_number_2017 globaldata-number_of_employees 0.005883737349964698
EDIT 52 companiesmarketcap-rank disfold-rank 0.0
EDIT 53 companiesmarketcap-rank valuetoday-rank 0.0
EDIT 54 companiesmarketcap-rank ft-revenue_2017_eu 0.8666666666666667
JACCARD 54 companiesmarketcap-rank ft-revenue_2017_eu 0.00010837758751490192
EDIT 55 companiesmarketcap-rank valuetoday-employees_number 0.9375
JACCARD 55 companiesmarke

...

EDIT 116 forbes-industry ambitiobox-name 0.875
JACCARD 116 forbes-industry ambitiobox-name 0.0001092896174863388
EDIT 117 forbes-industry ariregister-name 0.875
JACCARD 117 forbes-industry ariregister-name 0.0006591957811470006
EDIT 118 forbes-industry cbinsights-name 0.875
JACCARD 118 forbes-industry cbinsights-name 0.0007961783439490446
EDIT 119 forbes-industry companiesmarketcap-name 0.875
JACCARD 119 forbes-industry companiesmarketcap-name 0.0
EDIT 120 forbes-industry disfold-name 0.875
JACCARD 120 forbes-industry disfold-name 0.0
EDIT 121 forbes-industry forbes-name 0.875
JACCARD 121 forbes-industry forbes-name 0.00047551117451260106
EDIT 122 forbes-industry ft-name 0.875
JACCARD 122 forbes-industry ft-name 0.0002380952380952381
EDIT 123 forbes-industry globaldata-name 0.875
JACCARD 123 forbes-industry globaldata-name 0.00018573551263001485
EDIT 124 forbes-industry govuk-name 0.875
JACCARD 124 forbes-industry govuk-name 0.0001444460494005489
EDIT 125 forbes-industry hithorizons-na

JACCARD 199 wikipedia-industry companiesmarketcap-name 0.0
EDIT 200 wikipedia-industry disfold-name 0.875
JACCARD 200 wikipedia-industry disfold-name 0.0
EDIT 201 wikipedia-industry forbes-name 0.875
JACCARD 201 wikipedia-industry forbes-name 0.00042211903756859433
EDIT 202 wikipedia-industry ft-name 0.875
JACCARD 202 wikipedia-industry ft-name 0.0002239140170174653
EDIT 203 wikipedia-industry globaldata-name 0.875
JACCARD 203 wikipedia-industry globaldata-name 0.00017699115044247788
EDIT 204 wikipedia-industry govuk-name 0.875
JACCARD 204 wikipedia-industry govuk-name 0.00013910140492418973
EDIT 205 wikipedia-industry hithorizons-name 0.875
JACCARD 205 wikipedia-industry hithorizons-name 0.0010183299389002036
EDIT 206 wikipedia-industry infoclipper-name 0.875
JACCARD 206 wikipedia-industry infoclipper-name 0.0002883506343713956
EDIT 207 wikipedia-industry teamblind-name 0.875
JACCARD 207 wikipedia-industry teamblind-name 0.0007930214115781126
EDIT 208 wikipedia-industry valuetoday-nam

JACCARD 28 companiesmarketcap-pricecap disfold-revenue 0.0005443658138268917
EDIT 29 companiesmarketcap-pricecap globaldata-revenue 0.75
JACCARD 29 companiesmarketcap-pricecap globaldata-revenue 0.0001785076758300607
EDIT 30 companiesmarketcap-pricecap valuetoday-revenue 0.75
JACCARD 30 companiesmarketcap-pricecap valuetoday-revenue 0.0002021018593371059
EDIT 31 companiesmarketcap-pricecap forbes-sales 0.875
JACCARD 31 companiesmarketcap-pricecap forbes-sales 0.1538334161284981
EDIT 32 companiesmarketcap-pricecap companiesmarketcap-earnings 1.0
JACCARD 32 companiesmarketcap-pricecap companiesmarketcap-earnings 0.0002021018593371059
EDIT 33 companiesmarketcap-pricecap forbes-market_value 0.8333333333333334
JACCARD 33 companiesmarketcap-pricecap forbes-market_value 0.1526544821583986
EDIT 34 disfold-market_cap globaldata-market_cap 0.0
EDIT 35 disfold-market_cap valuetoday-annual_net_income_usd 0.7619047619047619
JACCARD 35 disfold-market_cap valuetoday-annual_net_income_usd 0.0001075037

JACCARD 26 companiesmarketcap-share_price forbes-sales 0.00011109876680368848
EDIT 27 companiesmarketcap-share_price companiesmarketcap-earnings 0.8181818181818182
JACCARD 27 companiesmarketcap-share_price companiesmarketcap-earnings 0.00014322543683758235
EDIT 28 companiesmarketcap-share_price forbes-market_value 0.6666666666666666
JACCARD 28 companiesmarketcap-share_price forbes-market_value 0.00011554015020219527
EDIT 29 companiesmarketcap-share_price valuetoday-annual_revenue_usd 0.8333333333333334
JACCARD 29 companiesmarketcap-share_price valuetoday-annual_revenue_usd 8.445945945945946e-05
EDIT 30 forbes-profit companiesmarketcap-pricecap 0.75
JACCARD 30 forbes-profit companiesmarketcap-pricecap 0.06992062151663571
EDIT 31 forbes-profit disfold-market_cap 0.9
JACCARD 31 forbes-profit disfold-market_cap 0.06678630944750673
EDIT 32 forbes-profit globaldata-market_cap 0.9
JACCARD 32 forbes-profit globaldata-market_cap 0.029356417011667295
EDIT 33 forbes-profit valuetoday-annual_net_i

JACCARD 99 companiesmarketcap-revenue valuetoday-annual_revenue_usd 0.00020580366330520683
EDIT 100 disfold-revenue globaldata-revenue 0.0
EDIT 101 disfold-revenue valuetoday-revenue 0.0
EDIT 102 disfold-revenue forbes-sales 0.8571428571428571
JACCARD 102 disfold-revenue forbes-sales 0.0011614401858304297
EDIT 103 disfold-revenue companiesmarketcap-earnings 0.875
JACCARD 103 disfold-revenue companiesmarketcap-earnings 0.0017667844522968198
EDIT 104 disfold-revenue forbes-market_value 0.5833333333333334
JACCARD 104 disfold-revenue forbes-market_value 0.0008936550491510277
EDIT 105 disfold-revenue valuetoday-annual_revenue_usd 0.6111111111111112
JACCARD 105 disfold-revenue valuetoday-annual_revenue_usd 0.00018436578171091445
EDIT 106 globaldata-revenue valuetoday-revenue 0.0
EDIT 107 globaldata-revenue forbes-sales 0.8571428571428571
JACCARD 107 globaldata-revenue forbes-sales 0.16965456930476608
EDIT 108 globaldata-revenue companiesmarketcap-earnings 0.875
JACCARD 108 globaldata-revenue

### TASK 4

In [ ]:
print("\nMake schema\n")
remove_and_make_dir(final_schema_schema)
make_schema_final(final_datasets_base_path, final_schema_final_dictionary, final_schema_schema, ".json")